# Implementing Convolutional Neural Network (CNN) with MNIST data

CNN is composed of following components:
- CNN: extracts features from an image
- Max Pooling: reduces features from an image
- Fully Connected Network: uses extracted and reduced features as input to perform downstream tasks.

## 1. Import Library

In [1]:
import torch
import torch.nn as nn # 신경망들이 포함됨
import torch.optim as optim # 최적화 알고리즘들이 포함됨
import torch.nn.init as init # 텐서에 초기값을 줌

import torchvision.datasets as datasets # 이미지 데이터셋 집합체
import torchvision.transforms as transforms # 이미지 변환 툴

from torch.utils.data import DataLoader # 학습 및 배치로 모델에 넣어주기 위한 툴

import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'torch'

## 2. Set Hyperparameter

- batch_size: